<a href="https://colab.research.google.com/github/nateanl/audio/blob/mvdr/examples/beamforming/MVDR_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is a tutorial on how to apply MVDR beamforming by using [torchaudio](https://github.com/pytorch/audio)
-----------

The multi-channel audio example is selected from [ConferencingSpeech](https://github.com/ConferencingSpeech/ConferencingSpeech2021) dataset. 

```
original filename: SSB07200001\#noise-sound-bible-0038\#7.86_6.16_3.00_3.14_4.84_134.5285_191.7899_0.4735\#15217\#25.16333303751458\#0.2101221178590021.wav
```

Note: 
- You need to clone the latest [torchaudio](https://github.com/pytorch/audio) repo in order to use the MVDR module under audio/examples/beamforming/mvdr.py
- You need to use the nightly torchaudio in order to use InverseSpectorgram transform module.


Steps

- Ideal Ratio Mask (IRM) is generated by dividing the clean/noise magnitude by the mixture magnitude.
- We test all three solutions (``ref_channel``, ``stv_evd``, ``stv_power``) of torchaudio's MVDR module.
- We test the single-channel and multi-channel masks for MVDR beamforming. The multi-channel mask is averaged along channel dimension when computing the covariance matrices of speech and noise, respectively.

In [ ]:
!pip install --pre torchaudio -f https://download.pytorch.org/whl/nightly/torch_nightly.html --force

In [ ]:
!git clone https://github.com/nateanl/torchaudio_mvdr_tutorial

In [ ]:
!git clone https://github.com/pytorch/audio.git

In [ ]:
import sys
sys.path.append('./audio/examples/')

In [ ]:
from beamforming.mvdr import MVDR
import torch
import torchaudio
import IPython.display as ipd

### Load audios of mixture, reverberated clean speech, and dry clean speech.

In [ ]:
mix, sr = torchaudio.load('./torchaudio_mvdr_tutorial/wavs/mix.wav')
reverb_clean, sr2 = torchaudio.load('./torchaudio_mvdr_tutorial/wavs/reverb_clean.wav')
clean, sr3 = torchaudio.load('./torchaudio_mvdr_tutorial/wavs/clean.wav')
assert sr == sr2
noise = mix - reverb_clean

## Note: The MVDR Module requires ``torch.cdouble`` dtype for noisy STFT. We need to convert the dtype of the waveforms to ``torch.double``

In [ ]:
mix = mix.to(torch.double)
noise = noise.to(torch.double)
clean = clean.to(torch.double)

### Initilize the Spectrogram and InverseSpectrogram modules

In [ ]:
stft = torchaudio.transforms.Spectrogram(n_fft=1024, hop_length=256, return_complex=True, power=None)
istft = torchaudio.transforms.InverseSpectrogram(n_fft=1024, hop_length=256)

### Compute the complex-valued STFT of mixture, clean speech, and noise

In [ ]:
spec_mix = stft(mix)
spec_clean = stft(clean)
spec_noise = stft(noise)

### Generate the Ideal Ratio Mask (IRM). Floor the mask value to be [0, 1]

In [ ]:
def get_irms(spec_clean, spec_noise, spec_mix):
    mag_mix = spec_mix.abs() ** 2
    mag_clean = spec_clean.abs() ** 2
    mag_noise = spec_noise.abs() ** 2
    irm_speech = mag_clean / (mag_mix + 1e-8)
    irm_noise = mag_noise / (mag_mix + 1e-8)
    irm_speech[irm_speech>=1] =1.
    irm_noise[irm_noise>=1] = 1.
    return irm_speech, irm_noise

In [ ]:
irm_speech, irm_noise = get_irms(spec_clean, spec_noise, spec_clean)

### Apply MVDR beamforming by using multi-channel masks

In [ ]:
results_multi = {}
for solution in ['ref_channel', 'stv_evd', 'stv_power']:
    mvdr = MVDR(ref_channel=0, solution=solution, multi_mask=True)
    stft_est = mvdr(spec_mix, irm_speech, irm_noise)
    est = istft(stft_est, length=mix.shape[-1])
    results_multi[solution] = est

### Apply MVDR beamforming by using single-channel masks 
(We use the 1st channel as an example. The channel selection may depend on the design of the microphone array)

In [ ]:
results_single = {}
for solution in ['ref_channel', 'stv_evd', 'stv_power']:
    mvdr = MVDR(ref_channel=0, solution=solution, multi_mask=False)
    stft_est = mvdr(spec_mix, irm_speech[0], irm_noise[0])
    est = istft(stft_est, length=mix.shape[-1])
    results_single[solution] = est

### Compute Si-SDR scores

In [ ]:
def si_sdr(estimate, reference, epsilon=1e-8):
    estimate = estimate - estimate.mean()
    reference = reference - reference.mean()
    reference_pow = reference.pow(2).mean(axis=1, keepdim=True)
    mix_pow = (estimate * reference).mean(axis=1, keepdim=True)
    scale = mix_pow / (reference_pow + epsilon)

    reference = scale * reference
    error = estimate - reference

    reference_pow = reference.pow(2)
    error_pow = error.pow(2)

    reference_pow = reference_pow.mean(axis=1)
    error_pow = error_pow.mean(axis=1)

    sisdr = 10 * torch.log10(reference_pow) - 10 * torch.log10(error_pow)
    return sisdr.item()

### Single-channel mask results

In [ ]:
for solution in results_single:
    print(solution+": ", si_sdr(results_single[solution][None,...], clean[0:1]))

### Multi-channel mask results

In [ ]:
for solution in results_multi:
    print(solution+": ", si_sdr(results_multi[solution][None,...], clean[0:1]))

### Display the mixture audio

In [ ]:
print("Mixture speech")
ipd.Audio(mix[0], rate=16000)

### Display the noise

In [ ]:
print("Noise")
ipd.Audio(noise[0], rate=16000)

### Display the clean speech

In [ ]:
print("Clean speech")
ipd.Audio(clean[0], rate=16000)

### Display the enhanced audios¶

In [ ]:
print("multi-channel mask, ref_channel solution")
ipd.Audio(results_multi['ref_channel'], rate=16000)

In [ ]:
print("multi-channel mask, stv_evd solution")
ipd.Audio(results_multi['stv_evd'], rate=16000)

In [ ]:
print("multi-channel mask, stv_power solution")
ipd.Audio(results_multi['stv_power'], rate=16000)

In [ ]:
print("single-channel mask, ref_channel solution")
ipd.Audio(results_single['ref_channel'], rate=16000)

In [ ]:
print("single-channel mask, stv_evd solution")
ipd.Audio(results_single['stv_evd'], rate=16000)

In [ ]:
print("single-channel mask, stv_power solution")
ipd.Audio(results_single['stv_power'], rate=16000)